## 数据型描述符、非数据型描述符、实例属性的访问优先级

In [1]:
class DataDescriptor:    
    def __init__(self, value):
        self.value = value
    
    def __get__(self, instance, owner):
        print('data descriptor __get__() is called.', instance, owner)
        return self.value
    
    def __set__(self, instance, value):
        print('data descriptor __set__() is called.', instance, value)
        self.value = value
        
class NonDataDescriptor:    
    def __init__(self, value):
        self.value = value
    
    def __get__(self, instance, owner):
        print('non-data descriptor __get__() is called.', instance, owner)
        return self.value

In [9]:
class Clazz:
    pass

x = Clazz()
x.n = DataDescriptor(200)
x.n

In [10]:
x.__dict__

{'n': <__main__.DataDescriptor at 0x33d9110>}

In [2]:
class Foo:
    d = DataDescriptor(10)

c = Foo()
c.d

data descriptor __get__() is called. <__main__.Foo object at 0x033C9970> <class '__main__.Foo'>


10

In [8]:
a = Foo()
a.d

data descriptor __get__() is called. <__main__.Foo object at 0x033C9CD0> <class '__main__.Foo'>


20

In [11]:
Foo.d

data descriptor __get__() is called. None <class '__main__.Foo'>


20

In [12]:
Foo.d = 1000

In [5]:
c.d = 20
c.d

data descriptor __set__() is called. <__main__.Foo object at 0x033C9970> 20
data descriptor __get__() is called. <__main__.Foo object at 0x033C9970> <class '__main__.Foo'>


20

In [3]:
class Spam:
    n = NonDataDescriptor(100)
    
s = Spam()
s.n

non-data descriptor __get__() is called. <__main__.Spam object at 0x033C9710> <class '__main__.Spam'>


100

In [4]:
s.n = 20
s.n

20

In [6]:
s.__dict__

{'n': 20}

In [7]:
Spam.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Spam' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Spam' objects>,
              'n': <__main__.NonDataDescriptor at 0x33c9730>})

https://www.jianshu.com/p/885d59db57fc

# `__getattribute__()`与`__getattr__()`

### 用作实例属性的获取和拦截

当访问某个实例属性时， getattribute会被无条件调用，如未实现自己的getattr方法，会抛出AttributeError提示找不到这个属性，如果自定义了自己getattr方法的话，方法会在这种找不到属性的情况下被调用，比如上面的例子中的情况。所以在找不到属性的情况下通过实现自定义的getattr方法来实现一些功能是一个不错的方式，因为它不会像getattribute方法每次都会调用可能会影响一些正常情况下的属性访问：

### 自定义getattribute的时候防止无限递归

### 同时覆盖掉getattribute和getattr的时候，在getattribute中需要模仿原本的行为抛出AttributeError或者手动调用getattr